In [0]:
spark.sql("SHOW TABLES IN default").show(truncate=False)


+--------+---------------------+-----------+
|database|tableName            |isTemporary|
+--------+---------------------+-----------+
|default |food_delivery_dataset|false      |
|default |shyam                |false      |
+--------+---------------------+-----------+



In [0]:
# Load your dataset from the table you created
orders_df = spark.table("default.food_delivery_dataset")

# Show first few rows
orders_df.show(5)


+--------+-----------+-------------+------------+-------------+---------------+-------------------+---------------------+-----------+-------------+------------+-------------------+
|order_id|customer_id|customer_name|    locality|restaurant_id|restaurant_name|delivery_partner_id|delivery_partner_name|order_value|delivery_time|order_status|     date_timestamp|
+--------+-----------+-------------+------------+-------------+---------------+-------------------+---------------------+-----------+-------------+------------+-------------------+
|    O001|       C105|       Vikram|        KPHB|         R205|Tandoori Flames|               D304|                Manoj|        783|           23|   cancelled|2025-08-05 18:19:00|
|    O002|       C109|         Ravi|BanjaraHills|         R201|  Biryani House|               D303|                Kavya|        738|           24|   delivered|2025-08-06 12:54:00|
|    O003|       C104|        Sneha|  HitechCity|         R205|Tandoori Flames|               D

In [0]:
from pyspark.sql.functions import sum

# Total order value per customer
customer_order_value = orders_df.groupBy("customer_id") \
                                .agg(sum("order_value").alias("total_order_value"))

# Show results
customer_order_value.show(10, truncate=False)


+-----------+-----------------+
|customer_id|total_order_value|
+-----------+-----------------+
|C109       |3309             |
|C102       |4362             |
|C106       |4319             |
|C107       |2418             |
|C104       |2626             |
|C105       |3886             |
|C101       |2834             |
|C103       |2367             |
|C108       |2270             |
|C110       |1814             |
+-----------+-----------------+



In [0]:
from pyspark.sql.functions import avg

# Average delivery time per restaurant
avg_delivery_time = orders_df.groupBy("restaurant_id") \
                             .agg(avg("delivery_time").alias("avg_delivery_time"))

avg_delivery_time.show(10, truncate=False)


+-------------+-----------------+
|restaurant_id|avg_delivery_time|
+-------------+-----------------+
|R202         |37.55555555555556|
|R203         |32.88235294117647|
|R204         |34.84615384615385|
|R205         |34.22222222222222|
|R201         |33.0             |
+-------------+-----------------+



In [0]:
# Register DataFrame as a temp view
orders_df.createOrReplaceTempView("orders")


In [0]:
# Count cancellations per customer
frequent_cancellations = spark.sql("""
    SELECT 
        customer_id,
        COUNT(*) AS total_cancellations
    FROM orders
    WHERE order_status = 'cancelled'
    GROUP BY customer_id
    HAVING COUNT(*) > 1
    ORDER BY total_cancellations DESC
""")

frequent_cancellations.show(10, truncate=False)


+-----------+-------------------+
|customer_id|total_cancellations|
+-----------+-------------------+
|C106       |5                  |
|C108       |4                  |
|C105       |3                  |
|C101       |3                  |
|C104       |3                  |
|C102       |3                  |
|C107       |3                  |
|C109       |2                  |
|C103       |2                  |
|C110       |2                  |
+-----------+-------------------+



In [0]:
orders_df.createOrReplaceTempView("orders")

peak_hours = spark.sql("""
    SELECT 
        HOUR(date_timestamp) AS order_hour,
        COUNT(*) AS total_orders
    FROM orders
    GROUP BY HOUR(date_timestamp)
    ORDER BY total_orders DESC
""")

peak_hours.show(10, truncate=False)


+----------+------------+
|order_hour|total_orders|
+----------+------------+
|22        |7           |
|23        |5           |
|21        |4           |
|7         |4           |
|9         |3           |
|2         |3           |
|17        |3           |
|3         |3           |
|10        |3           |
|11        |3           |
+----------+------------+
only showing top 10 rows


In [0]:
peak_hours = spark.sql("""
    SELECT HOUR(date_timestamp) AS order_hour,
           COUNT(*) AS total_orders
    FROM orders
    GROUP BY HOUR(date_timestamp)
    ORDER BY total_orders DESC
""")
peak_hours.show()


+----------+------------+
|order_hour|total_orders|
+----------+------------+
|        22|           7|
|        23|           5|
|         7|           4|
|        21|           4|
|         2|           3|
|        11|           3|
|         3|           3|
|        18|           3|
|         9|           3|
|        14|           3|
|        10|           3|
|        17|           3|
|         1|           2|
|         8|           2|
|        13|           2|
|        19|           2|
|        20|           2|
|         6|           1|
|        16|           1|
|        15|           1|
+----------+------------+
only showing top 20 rows
